# Creating topics for the cases
This notebooks explores the topics we can create from the cases. It's gonna involve a lot of trial and error, but it will hopefully be well-documented

In [16]:
import pickle
import requests
import random
import pandas as pd
import numpy as np
from pathlib import Path
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def pickle_object(obj, file_path):
    with open(file_path, "wb") as f:
        pickle.dump(obj, f)

def flatten_embeddings(embedding_dict):
    """ Creates a big matrix with all the embeddings from the dict """
    return np.vstack(embedding_dict.values())

def flatten_list(lst):
    return [elem for sublist in lst for elem in sublist]

def get_paragraphs(paragraph_dict):
    return flatten_list(list(paragraph_dict.values()))


def load_text_url(text_url: str): 
    """Inputs a URL of a newline seperated text-file and returns a list"""
    response = requests.get(text_url)
    return response.text.split("\n")

## Loading the data

In [18]:
DATA_DIR = Path("../../BscThesisData/data")
MODEL_PATH = Path("../models")
embedding_dict = read_pickle(DATA_DIR / "embedding_dict.pkl")
clean_paragraphs = read_pickle(DATA_DIR / "paragraph_dict.pkl")

Now it's time to extract the embeddings for BERTopic to process

In [19]:
embeddings = flatten_embeddings(embedding_dict)
docs = get_paragraphs(clean_paragraphs)
print(random.sample(docs, 5))

['Retten fandt ikke, at SKATs afgørelse var i strid med officialprincippet. Retten tiltrådte desuden det af Skatteministeriet anførte, hvorefter SKATs afgørelse var udtryk for en særdeles lempelig skønsmæssig ansættelse af sagsøgeren. Da sagsøgeren ikke i øvrigt havde påvist, at SKATs skøn var åbenbart urimeligt eller hvilede på et forkert grundlag, blev ministeriet frifundet.', 'Skatteministeriet blev som følge heraf i det hele frifundet.', 'Højesteret fandt, at udgifterne måtte anses at skulle afholdes af det præsterede, ansvarlige indskud. Udgifternes afholdelse ville derfor reducere den til anparterne hørende hæftelse. Højesteret fandt det ikke godtgjort, at afholdelsen af disse udgifter reelt modsvaredes af et aktiv for kommanditselskabet, hvorfor udgifterne i medfør af cirkulærets pkt. k skulle fradrages ved opgørelsen af fradragskontoen. Det forhold, at kommanditisten regnskabsmæssigt havde behandlet udgifterne, således at de ikke påvirkede hans kapitalkonto i kommanditselskabet

### Now we initialize the models

# Load Cleaning Models

In [20]:
STOP_WORD_URL = "https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt"
STOP_WORDS = load_text_url(STOP_WORD_URL)
vectorizer_model = CountVectorizer(stop_words=STOP_WORDS)
pickle_object(vectorizer_model, MODEL_PATH / "vectorizer.pkl")

In [21]:
# Powering up the transformer!
topic_model = BERTopic("Maltehb/-l-ctra-danish-electra-small-cased", vectorizer_model=vectorizer_model, nr_topics=5)

In [22]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [23]:
topic_model.visualize_topics()

In [24]:
preds_df = pd.DataFrame(list(zip(topics, probs, docs)), columns = ["topic", "prob", "doc"])
preds_df.to_csv(DATA_DIR / "doc_topics.csv", index=False)

In [32]:
topic_dict = topic_model.get_topics()
topic_dict_clean = {k: [tup[0] for tup in word_list] for k, word_list in topic_dict.items() if k != -1}
pickle_object(topic_dict_clean, DATA_DIR / "topic_dict.pkl")

In [25]:
# Saving the model
topic_model.save(str(MODEL_PATH / "topic_model"), save_embedding_model=False)

C:\Users\jhr\Anaconda3\envs\bertopic_explore\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

